## Reproduce solvent segregation in Wang 2022 - Fig. 4
### $ D_{\text{e},\text{EC}}^0 = 0, \, 2.5 \times 10^{-12}, \, 2.5 \times 10^{-11} \, \text{m}^2/\text{s} $

In [1]:
# magic lines that avoid re-start
%load_ext autoreload
%autoreload 2
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp,sqrt;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [2]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pybamm.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip/Rio_Code/Fun_P3"))
sys.path.append(str_path_1) 
from Fun_P3 import *
BasicPath = os.path.expanduser("~/EnvPBGEM_ECDrag2/Double_SimSave/Double_Trans/")
Target  = 'Rate_Performance/' 
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)
Save_Fig = True

In [3]:
# define parameter of double and single 
Para_dict_Same = {
   "Mesh list":[ [20,10,20,100,20], ],   # Simon uses 30
   "Para_Set":[ "Li2023_ECdrag",],
   "Contact resistance [Ohm]":[12e-3],
   "Initial Neg SOC":[0.8841301667966484,],
   "Initial Pos SOC":[0.23552755074598045,],}
Para_dict_DD_ONLY = {
   "Model option":[{
      "SEI": "constant",
      "SEI film resistance":"distributed",   
      "SEI porosity change": "true",
      "solvent diffusion": "double spatial consume w refill",
      "electrolyte conductivity": "sol full"  ,
     "contact resistance":"true",  },],
   "Lithium ion EC cross diffusivity [m2.s-1]":[0,2.5E-12,2.5e-11,], # 
}
Para_dict_SD_ONLY = {
   "Model option":[{
      "SEI": "constant",
      "SEI film resistance":"distributed",   
      "SEI porosity change": "true",
      "solvent diffusion": "single no consume wo refill",
      "electrolyte conductivity": "full"  ,
      "contact resistance":"true",  },],
   "Lithium ion EC cross diffusivity [m2.s-1]":[0,], #  
   "Measured dLJP_dce":["dLJP_1_Specie_dc_e_np",]
}
Para_dict_DD = {**Para_dict_Same,**Para_dict_DD_ONLY}
Para_dict_SD = {**Para_dict_Same,**Para_dict_SD_ONLY}
Para_DD = [];  Para_SD=[];
recursive_scan(Para_DD,Para_dict_DD, list(Para_dict_DD.keys()), {})
recursive_scan(Para_SD,Para_dict_SD, list(Para_dict_SD.keys()), {})
print(len(Para_DD),len(Para_SD))

Path_pack = [BasicPath,Target,]

3 1


#### 2.5e-11 + 2.6C takes 40+ minutes and has spikes; 1e-11 + 2.8C takes 44 minutes, -> both not good

In [4]:
Rate_Dis_All = [1,1.5,2,2.5,2.525,2.55,2.575,2.6]     # np.arange(1,2.5+0.5,0.5) # np.arange(3.8,4.5+0.175,0.175)
print(Rate_Dis_All,len(Rate_Dis_All))

[1, 1.5, 2, 2.5, 2.525, 2.55, 2.575, 2.6] 8


In [5]:
for i in range(len(Para_DD)):  
    for Rate_Dis in Rate_Dis_All:
        D_e_EC_cross = Para_DD[i]["Lithium ion EC cross diffusivity [m2.s-1]"]
        str_pickle =  f"Double_D_e_EC_cross={D_e_EC_cross}"
        sol_return = Run_P3_OneCycle_Dict(
            i, Para_DD[i], Path_pack,Rate_Dis,str_pickle,
            Return_sol=False,Save_sol=True)

Start Now! C rate: 1


CasADi - 2024-01-31 10:19:52 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:515]


Finish Double_D_e_EC_cross=0_Rate=1C with Spent 53.499 s
Start Now! C rate: 1.5
Finish Double_D_e_EC_cross=0_Rate=1.5C with Spent 1 minute, 38 seconds
Start Now! C rate: 2


At t = 0.00886996, , mxstep steps taken before reaching tout.


Finish Double_D_e_EC_cross=0_Rate=2C with Spent 2 minutes, 51 seconds
Start Now! C rate: 2.5
Finish Double_D_e_EC_cross=0_Rate=2.5C with Spent 1 minute, 9 seconds
Start Now! C rate: 2.525
Finish Double_D_e_EC_cross=0_Rate=2.525C with Spent 1 minute, 8 seconds
Start Now! C rate: 2.55
Finish Double_D_e_EC_cross=0_Rate=2.55C with Spent 1 minute, 9 seconds
Start Now! C rate: 2.575
Finish Double_D_e_EC_cross=0_Rate=2.575C with Spent 1 minute, 20 seconds
Start Now! C rate: 2.6


...

Finish Double_D_e_EC_cross=2.5e-11_Rate=2.55C with Spent 18 minutes, 29 seconds
Start Now! C rate: 2.575


At t = 0.000559398, , mxstep steps taken before reaching tout.
At t = 0.000559398, , mxstep steps taken before reaching tout.
At t = 0.000559398, , mxstep steps taken before reaching tout.
At t = 0.000559398, , mxstep steps taken before reaching tout.
At t = 0.000559398, , mxstep steps taken before reaching tout.
At t = 0.000254389, , mxstep steps taken before reaching tout.
At t = 0.000254382, , mxstep steps taken before reaching tout.
At t = 0.000254381, , mxstep steps taken before reaching tout.
At t = 0.00025438, , mxstep steps taken before reaching tout.
At t = 0.000254358, , mxstep steps taken before reaching tout.
At t = 0.000254358, , mxstep steps taken before reaching tout.


Finish Double_D_e_EC_cross=2.5e-11_Rate=2.575C with Spent 19 minutes, 38 seconds
Start Now! C rate: 2.6


At t = 0.00317959, , mxstep steps taken before reaching tout.
At t = 0.00317959, , mxstep steps taken before reaching tout.
At t = 0.000666973, , mxstep steps taken before reaching tout.
At t = 0.000666973, , mxstep steps taken before reaching tout.
At t = 0.000666973, , mxstep steps taken before reaching tout.
At t = 0.000666973, , mxstep steps taken before reaching tout.
At t = 4.42745e-05, , mxstep steps taken before reaching tout.
At t = 4.42739e-05, , mxstep steps taken before reaching tout.
At t = 4.4274e-05, , mxstep steps taken before reaching tout.
At t = 4.4274e-05, , mxstep steps taken before reaching tout.
At t = 7.45762e-06, , mxstep steps taken before reaching tout.
At t = 7.45803e-06, , mxstep steps taken before reaching tout.
At t = 7.52818e-06, , mxstep steps taken before reaching tout.
At t = 2.56295e-06, , mxstep steps taken before reaching tout.
At t = 6.59828e-08, , mxstep steps taken before reaching tout.
At t = 6.25387e-08, , mxstep steps taken before reaching to

Finish Double_D_e_EC_cross=2.5e-11_Rate=2.6C with Spent 36 minutes, 26 seconds


In [6]:
for i in range(len(Para_SD)):  
    for Rate_Dis in Rate_Dis_All:
        D_e_EC_cross = Para_SD[i]["Lithium ion EC cross diffusivity [m2.s-1]"]
        str_pickle =  f"Single"
        sol_return = Run_P3_OneCycle_Dict(
            i, Para_DD[i], Path_pack,Rate_Dis,str_pickle,
            Return_sol=False,Save_sol=True)

Start Now! C rate: 1
Finish Single_Rate=1C with Spent 58.906 s
Start Now! C rate: 1.5
Finish Single_Rate=1.5C with Spent 2 minutes, 23 seconds
Start Now! C rate: 2


At t = 0.00886996, , mxstep steps taken before reaching tout.


Finish Single_Rate=2C with Spent 3 minutes, 26 seconds
Start Now! C rate: 2.5
Finish Single_Rate=2.5C with Spent 1 minute, 25 seconds
Start Now! C rate: 2.525
Finish Single_Rate=2.525C with Spent 1 minute, 26 seconds
Start Now! C rate: 2.55
Finish Single_Rate=2.55C with Spent 1 minute, 28 seconds
Start Now! C rate: 2.575
Finish Single_Rate=2.575C with Spent 1 minute, 26 seconds
Start Now! C rate: 2.6


At t = 0.00897636, , mxstep steps taken before reaching tout.


Finish Single_Rate=2.6C with Spent 1 minute, 33 seconds
